## import libraries

In [1]:
# Libraries and setup

# Auto reload changes
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../") # go to parent dir

from src.utils.dependencies import *
# from src.utils.utility import Utility
# utl = Utility()

# log file schema
from src.log_file_schema import *

# data handler
from src.data_handler import DataHandler
dh = DataHandler()
n=5

/Users/amishra/github/paypay_avinash/DataEngineerChallenge/src/utils/libraries.py:16: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


## Create or Get Spark session

In [2]:
#create or get spark session
spark = SparkSession.builder \
    .master("local[*]")\
    .appName("PayPayChallenge")\
    .getOrCreate()

## log file locationn

In [3]:
def duration(start, end):
        try:
            num_of_seconds = (end - start).total_seconds()
        except:
            num_of_seconds = 0
        return num_of_seconds
get_duration = udf(duration, FloatType())

In [4]:
def average_session_time(df_ip_session):
    window_func_session = Window.partitionBy("ip_session_count").orderBy("current_timestamp")
    df_session = df_ip_session.withColumn("previous_timestamp_session",
                              lag(df_ip_session["current_timestamp"]).over(window_func_session)) \
                  .withColumn("current_session_duration",
                              get_duration(col("previous_timestamp_session"), col("current_timestamp")))
    df_session_total = df_session.groupby("ip_session_count").agg(
            sum("current_session_duration").alias("total_session_time")).cache()
    df_session_avg = df_session_total.select([mean("total_session_time").alias("avg_session_time")]).cache()
    df_session_avg.show()
    return df_session


def count_unique_request(df_session):
    df_unique_url = df_session.groupby("ip_session_count").agg(
        countDistinct("request_url").alias("count_unique_requests"))
    df_unique_url.show()

def get_longest_session_time(df_session):
    df_ip_time = df_session.groupby("client_ip").agg(
                sum("session_duration").alias("session_time_all"),
                count("client_ip").alias("num_sessions"),
                max("session_duration").alias("session_duration_max")) \
          .withColumn("avg_session_time", col("session_time_all") / col("num_sessions")) \
          .orderBy(col("avg_session_time"), ascending=False)
    df_ip_time.show()


num_partitions = 15
session_time = 15*60
def solve(spark):
    df_logs = dh.initialization(spark).cache()
    # print(df_logs.limit(5).toPandas().head(5))
    df_ip_session = dh.preprocess(df_logs, gd=get_duration).repartition(num_partitions).cache()
    df_session = average_session_time(df_ip_session).repartition(num_partitions).cache()
    count_unique_request(df_session)
    get_longest_session_time(df_session)

In [5]:
solve(spark=spark)